In [1]:
import time
import datetime
import requests
import json
import pandas as pd

import dash
from dash import dash_table
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline

<function cufflinks.offline.go_offline(connected=None)>

In [2]:
#query = """query {
#    saleAuction(id: 2328) {
#    id
#    endedAt
#    endingPrice
#    }
#}"""

query = """query {
  saleAuctions(first:1000 orderBy: endedAt orderDirection: desc 
  		where: {
        open: false
        purchasePrice_not: null
      }
  		
  
  		) {
    id
    tokenId {
      id
      rarity
      generation
      mainClass
      subClass
      statBoost1
      statBoost2
      profession
      summons
      maxSummons
    }
  	endedAt
    purchasePrice
  }
}
"""

url = "https://graph.defikingdoms.com/subgraphs/name/defikingdoms/apiv5"
r = requests.post(url, json={"query": query})

In [3]:
if r.status_code == 200:
    print(json.dumps(r.json(), indent=2))
else:
    raise Exception(f"Query failed to run with a {r.status_code}.")

{
  "data": {
    "saleAuctions": [
      {
        "endedAt": "1636253184",
        "id": "15973",
        "purchasePrice": "220000000000000000000",
        "tokenId": {
          "generation": 1,
          "id": "9054",
          "mainClass": "Monk",
          "maxSummons": 10,
          "profession": "mining",
          "rarity": 1,
          "statBoost1": "DEX",
          "statBoost2": "AGI",
          "subClass": "Thief",
          "summons": 3
        }
      },
      {
        "endedAt": "1636253098",
        "id": "15832",
        "purchasePrice": "219000000000000000000",
        "tokenId": {
          "generation": 2,
          "id": "12821",
          "mainClass": "Monk",
          "maxSummons": 7,
          "profession": "fishing",
          "rarity": 1,
          "statBoost1": "AGI",
          "statBoost2": "AGI",
          "subClass": "Knight",
          "summons": 0
        }
      },
      {
        "endedAt": "1636253061",
        "id": "16011",
        "purchasePrice":

In [4]:
json_data = json.loads(r.text)

currentTime = datetime.datetime.utcnow()

In [5]:
df_data = json_data['data']['saleAuctions']
df = pd.DataFrame(df_data)

In [6]:
df.head()

,endedAt,id,purchasePrice,tokenId
0,1636253184,15973,220000000000000000000,"{'generation': 1, 'id': '9054', 'mainClass': '..."
1,1636253098,15832,219000000000000000000,"{'generation': 2, 'id': '12821', 'mainClass': ..."
2,1636253061,16011,170000000000000000000,"{'generation': 3, 'id': '13595', 'mainClass': ..."
3,1636252973,16046,189000000000000000000,"{'generation': 3, 'id': '12801', 'mainClass': ..."
4,1636252881,15957,299000000000000000000,"{'generation': 1, 'id': '9449', 'mainClass': '..."


In [7]:
df['tokenId'].apply(pd.Series)

,generation,id,mainClass,maxSummons,profession,rarity,statBoost1,statBoost2,subClass,summons
0,1,9054,Monk,10,mining,1,DEX,AGI,Thief,3
1,2,12821,Monk,7,fishing,1,AGI,AGI,Knight,0
2,3,13595,Pirate,6,mining,1,DEX,AGI,Thief,0
3,3,12801,Archer,7,gardening,1,VIT,WIS,Summoner,0
4,1,9449,Knight,10,mining,0,LCK,LCK,Thief,3
...,...,...,...,...,...,...,...,...,...,...
995,2,11431,Archer,9,fishing,0,STR,END,Knight,2
996,2,10966,Thief,9,foraging,2,VIT,AGI,Thief,2
997,1,10954,Wizard,10,mining,0,STR,INT,Pirate,3
998,2,9041,Thief,5,mining,1,AGI,AGI,Wizard,2


In [8]:
df2 = df['tokenId'].apply(pd.Series)

In [9]:
df2 = pd.concat([df2, df['purchasePrice']], axis=1)
df2 = pd.concat([df2, df['endedAt']], axis=1)
df2.head()

,generation,id,mainClass,maxSummons,profession,rarity,statBoost1,statBoost2,subClass,summons,purchasePrice,endedAt
0,1,9054,Monk,10,mining,1,DEX,AGI,Thief,3,220000000000000000000,1636253184
1,2,12821,Monk,7,fishing,1,AGI,AGI,Knight,0,219000000000000000000,1636253098
2,3,13595,Pirate,6,mining,1,DEX,AGI,Thief,0,170000000000000000000,1636253061
3,3,12801,Archer,7,gardening,1,VIT,WIS,Summoner,0,189000000000000000000,1636252973
4,1,9449,Knight,10,mining,0,LCK,LCK,Thief,3,299000000000000000000,1636252881


In [10]:
#df2 = pd.concat([df['tokenId'].apply(pd.Series), df['purchasePrice']], axis=1)
#df2 = pd.concat([df['tokenId'].apply(pd.Series), df['endedAt']], axis=1)

In [11]:
cols = ['id', 'rarity', 'generation', 'mainClass', 'subClass', 'statBoost1', 'statBoost2', 'profession', 'summons', 'maxSummons', 'purchasePrice', 'endedAt']

In [12]:
df2 = df2.reindex(columns=cols)

In [13]:
df2['rarity'] = df2['rarity'].replace([0, 1, 2, 3, 4], ['common', 'uncommon', 'rare', 'legendary', 'mythic'])
#print(df2)

In [14]:
df2.head()

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,purchasePrice,endedAt
0,9054,uncommon,1,Monk,Thief,DEX,AGI,mining,3,10,220000000000000000000,1636253184
1,12821,uncommon,2,Monk,Knight,AGI,AGI,fishing,0,7,219000000000000000000,1636253098
2,13595,uncommon,3,Pirate,Thief,DEX,AGI,mining,0,6,170000000000000000000,1636253061
3,12801,uncommon,3,Archer,Summoner,VIT,WIS,gardening,0,7,189000000000000000000,1636252973
4,9449,common,1,Knight,Thief,LCK,LCK,mining,3,10,299000000000000000000,1636252881


In [15]:
#drop empty values from purchasePrice

df2.dropna(subset=['purchasePrice'])

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,purchasePrice,endedAt
0,9054,uncommon,1,Monk,Thief,DEX,AGI,mining,3,10,220000000000000000000,1636253184
1,12821,uncommon,2,Monk,Knight,AGI,AGI,fishing,0,7,219000000000000000000,1636253098
2,13595,uncommon,3,Pirate,Thief,DEX,AGI,mining,0,6,170000000000000000000,1636253061
3,12801,uncommon,3,Archer,Summoner,VIT,WIS,gardening,0,7,189000000000000000000,1636252973
4,9449,common,1,Knight,Thief,LCK,LCK,mining,3,10,299000000000000000000,1636252881
...,...,...,...,...,...,...,...,...,...,...,...,...
995,11431,common,2,Archer,Knight,STR,END,fishing,2,9,195000000000000000000,1636141121
996,10966,rare,2,Thief,Thief,VIT,AGI,foraging,2,9,225000000000000000000,1636141119
997,10954,common,1,Wizard,Pirate,STR,INT,mining,3,10,225000000000000000000,1636141021
998,9041,uncommon,2,Thief,Wizard,AGI,AGI,mining,2,5,286980000000000000000,1636140965


In [16]:
#remove extra characters from jewel sold price
soldPrice = []

for x in df['purchasePrice']:
    for y in x:
        priceLen = len(x)-16
    x = x[: priceLen]
    x = int(float(x))/100
    soldPrice.append(x)


In [17]:
df2['soldPrice'] = soldPrice
df2 = df2.drop(['purchasePrice'], axis=1)

In [18]:
df2.head()

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,endedAt,soldPrice
0,9054,uncommon,1,Monk,Thief,DEX,AGI,mining,3,10,1636253184,220.0
1,12821,uncommon,2,Monk,Knight,AGI,AGI,fishing,0,7,1636253098,219.0
2,13595,uncommon,3,Pirate,Thief,DEX,AGI,mining,0,6,1636253061,170.0
3,12801,uncommon,3,Archer,Summoner,VIT,WIS,gardening,0,7,1636252973,189.0
4,9449,common,1,Knight,Thief,LCK,LCK,mining,3,10,1636252881,299.0


In [19]:
genstr = []

for x in df2['generation']:
    x = str(x)
    genstr.append(x)
    
df2['generationStr'] = genstr

In [20]:
utcTime = []

for x in df['endedAt']:
    x = int(x)
    x = datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')
    utcTime.append(x)
    #print(utcTime)
    
df2['timeStamp'] = utcTime
df2 = df2.drop(['endedAt'], axis=1)

In [22]:
#filter

#base data
warrior = df2

warriorC = df2[(df2["rarity"]=='common')]
warriorU = df2[(df2["rarity"]=='uncommon')]
warriorR = df2[(df2["rarity"]=='rare')]
warriorL = df2[(df2["rarity"]=='legendary')]
warriorM = df2[(df2["rarity"]=='mythic')]

#table data - drop 'generationStr' for readability
knight = df2
knight = knight.drop(['generationStr'], axis=1)


In [23]:
df2.to_excel("tavern_data.xlsx", index=False)

In [24]:
#sns.set_style("whitegrid")

#fig, ax = plt.subplots()
#fig.set_size_inches(15, 15)
#sns.boxplot(x="mainClass", y="soldPrice", data=fishing, hue='rarity', hue_order=['common', 'uncommon', 'rare', 'legendary', 'mythic'], palette='rainbow')

#fig.tight_layout()
#plt.savefig('save_as_a_png.png')

In [25]:
fig = go.Figure()

# px.scatter(warriorC, x="timeStamp", y="soldPrice",
#                 hover_name="id", hover_data={'rarity'})])

fig.add_trace(go.Scatter(x=warriorC.timeStamp, y=warriorC.soldPrice, mode='markers', name='Common',
                         hovertemplate=
                         '<b>ID</b>: %{text}<br>' +
                         '<b>Price</b>: %{y} Jewels' +
                         '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
                         text=warriorC['id'] + '<br>' +
                              '<b>Rarity</b>: ' + warriorC['rarity'] + '<br>' +
                              '<b>Generation</b>: ' + warriorC['generationStr'] + '<br>' + '<br>' +
                              '<b>Main Class</b>: ' + warriorC['mainClass'] + '<br>' +
                              '<b>Sub Class</b>: ' + warriorC['subClass'] + '<br>' +
                              '<b>Primary Boost</b>: ' + warriorC['statBoost1'] + '<br>' +
                              '<b>Secondary Boost</b>: ' + warriorC['statBoost2'] + '<br>' +
                              '<b>Profession</b>: ' + warriorC['profession'] + '<br>',
                         marker=dict(color='rgba(219, 217, 222, 1)', size=7)

                         ))

fig.add_trace(go.Scatter(x=warriorU.timeStamp, y=warriorU.soldPrice, mode='markers', name='Uncommon',
                         hovertemplate=
                         '<b>ID</b>: %{text}<br>' +
                         '<b>Price</b>: %{y} Jewels' +
                         '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
                         text=warriorU['id'] + '<br>' +
                              '<b>Rarity</b>: ' + warriorU['rarity'] + '<br>' +
                              '<b>Generation</b>: ' + warriorU['generationStr'] + '<br>' + '<br>' +
                              '<b>Main Class</b>: ' + warriorU['mainClass'] + '<br>' +
                              '<b>Sub Class</b>: ' + warriorU['subClass'] + '<br>' +
                              '<b>Primary Boost</b>: ' + warriorU['statBoost1'] + '<br>' +
                              '<b>Secondary Boost</b>: ' + warriorU['statBoost2'] + '<br>' +
                              '<b>Profession</b>: ' + warriorU['profession'] + '<br>',
                         marker=dict(color='rgba(115, 191, 131, 1)', size=7)
                         ))

fig.add_trace(go.Scatter(x=warriorR.timeStamp, y=warriorR.soldPrice, mode='markers', name='Rare',
                         hovertemplate=
                         '<b>ID</b>: %{text}<br>' +
                         '<b>Price</b>: %{y} Jewels' +
                         '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
                         text=warriorR['id'] + '<br>' +
                              '<b>Rarity</b>: ' + warriorR['rarity'] + '<br>' +
                              '<b>Generation</b>: ' + warriorR['generationStr'] + '<br>' + '<br>' +
                              '<b>Main Class</b>: ' + warriorR['mainClass'] + '<br>' +
                              '<b>Sub Class</b>: ' + warriorR['subClass'] + '<br>' +
                              '<b>Primary Boost</b>: ' + warriorR['statBoost1'] + '<br>' +
                              '<b>Secondary Boost</b>: ' + warriorR['statBoost2'] + '<br>' +
                              '<b>Profession</b>: ' + warriorR['profession'] + '<br>',
                         marker=dict(color='rgba(53, 147, 183, 1)', size=7)
                         ))

fig.add_trace(go.Scatter(x=warriorL.timeStamp, y=warriorL.soldPrice, mode='markers', name='Legendary',
                         hovertemplate=
                         '<b>ID</b>: %{text}<br>' +
                         '<b>Price</b>: %{y} Jewels' +
                         '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
                         text=warriorL['id'] + '<br>' +
                              '<b>Rarity</b>: ' + warriorL['rarity'] + '<br>' +
                              '<b>Generation</b>: ' + warriorL['generationStr'] + '<br>' + '<br>' +
                              '<b>Main Class</b>: ' + warriorL['mainClass'] + '<br>' +
                              '<b>Sub Class</b>: ' + warriorL['subClass'] + '<br>' +
                              '<b>Primary Boost</b>: ' + warriorL['statBoost1'] + '<br>' +
                              '<b>Secondary Boost</b>: ' + warriorL['statBoost2'] + '<br>' +
                              '<b>Profession</b>: ' + warriorL['profession'] + '<br>',
                         marker=dict(color='rgba(255, 164, 62, 1)', size=7)
                         ))

fig.add_trace(go.Scatter(x=warriorM.timeStamp, y=warriorM.soldPrice, mode='markers', name='Mythic',
                         hovertemplate=
                         '<b>ID</b>: %{text}<br>' +
                         '<b>Price</b>: %{y} Jewels' +
                         '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
                         text=warriorM['id'] + '<br>' +
                              '<b>Rarity</b>: ' + warriorM['rarity'] + '<br>' +
                              '<b>Generation</b>: ' + warriorM['generationStr'] + '<br>' + '<br>' +
                              '<b>Main Class</b>: ' + warriorM['mainClass'] + '<br>' +
                              '<b>Sub Class</b>: ' + warriorM['subClass'] + '<br>' +
                              '<b>Primary Boost</b>: ' + warriorM['statBoost1'] + '<br>' +
                              '<b>Secondary Boost</b>: ' + warriorM['statBoost2'] + '<br>' +
                              '<b>Profession</b>: ' + warriorM['profession'] + '<br>',
                         marker=dict(color='rgba(178, 109, 216, 1)', size=7)
                         ))

fig.update_traces(marker=dict(line=dict(width=.5)))
fig.update_layout(title='Tavern Sales - Last 1000 Heroes Sold',
                  titlefont=dict(family='Arial', size=24),
                  xaxis=dict(showgrid=True, ticks='outside'),
                  xaxis_title='Date in UTC',
                  yaxis_title='Jewel',
                  plot_bgcolor='white'
                  )

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

In [26]:
#drop unneeded columns
# df2 = df2.drop(['generationStr'], axis=1)
# warrior = warrior.drop(['generationStr'], axis=1)

In [34]:
#Initialize
# Setup the style from the link:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# Embed the style to the dashabord:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
#server = app.server

PAGE_SIZE = 25

app.layout = html.Div(
    children=[
        html.H1(children="DeFi Kingdom Tavern Dashboards",),
        html.Div(
            children="Random playground for various tavern dashboards."),
            
        html.Div(
            children="Data last updated: {}.".format(currentTime)),

        html.Div(
            children=[
                html.Div(children = 'Main Class', style={'fontSize': "16px", 'width': '50%'},className = 'menu-title'),
                dcc.Dropdown(
                    id = 'main-class',
                    options = [
                        {'label': MainClass, 'value':MainClass}
                        for MainClass in warrior.mainClass.sort_values().unique()
                    ], #'warrior' is the filter
                    clearable = True,
                    searchable = False,
                    className = 'dropdown', style={'fontSize': "14px",'textAlign': 'center'},
                ),
            ],
            className = 'menu',
        ), #the dropdown function
        
        dcc.RangeSlider(
            id='gen-slider', # any name you'd like to give it
            marks={
                0: '0',     # key=position, value=what you see
                1: '1',
                2: '2',
                3: '3',
                4: '4',
                5: '5',
                6: '6',
                7: '7',
                8: '8',
                9: '9',
                10: '10',
                11: '11',
            },
            step=1,                # number of steps between values
            min=0,
            max=11,
            value=[0,11],     # default value initially chosen
            dots=True,             # True, False - insert dots, only when step>1
            allowCross=False,      # True,False - Manage handle crossover
            disabled=False,        # True,False - disable handle
            #pushable=1,            # any number, or True with multiple handles
            #updatemode='drag',  # 'mouseup', 'drag' - update value method
            #included=True,         # True, False - highlight handle
            #vertical=False,        # True, False - vertical, horizontal slider
            #verticalHeight=900,    # hight of slider (pixels) when vertical=True
            className='None',
            tooltip={'always visible':False,  # show current slider values
                     'placement':'bottom'},
            ),
        
        dcc.Graph(id='main-chart', figure=fig),
        
        dash_table.DataTable(id='main-table',
        columns=[{"name": i, "id": i} for i in knight.columns],
        data=knight.to_dict('records'),
        page_current=0,
        page_size=PAGE_SIZE),
    
        
        html.Div(children="Tip jar: 0x71C52444b34fb9d99b3F3E0bD29084ba0EEe0436"),

        html.Div(children="Tips are appreciated :D"),
    ]
)

@app.callback(
    [Output("main-table", "data")],
    [Input("main-class", "value"),
     Input("gen-slider", "value")]
)

def update_tables(option_selected, gen_slider):
    if option_selected is None:
        filtered_df = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_df = filtered_df.drop(['generationStr'], axis=1)
    else:
        filtered_df = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_df = filtered_df[filtered_df['mainClass']==option_selected]
        filtered_df = filtered_df.drop(['generationStr'], axis=1)
    return [filtered_df.to_dict('records')]

@app.callback(
    Output("main-chart", "figure"),
    [Input("main-class", "value"),
     Input("gen-slider", "value")]
)

def update_charts(option_selected, gen_slider):
    if option_selected is None:
        filtered_dataC = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataU = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataR = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataL = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataM = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
    
        filtered_dataC = filtered_dataC[(filtered_dataC['rarity']=='common')]
        filtered_dataU = filtered_dataU[(filtered_dataU['rarity']=='uncommon')]
        filtered_dataR = filtered_dataR[(filtered_dataR['rarity']=='rare')]
        filtered_dataL = filtered_dataL[(filtered_dataL['rarity']=='legendary')]
        filtered_dataM = filtered_dataM[(filtered_dataM['rarity']=='mythic')]
    else:
        filtered_dataC = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataU = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataR = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataL = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        filtered_dataM = warrior[(warrior['generation']>=gen_slider[0]) & (warrior['generation']<=gen_slider[1])]
        
        filtered_dataC = filtered_dataC[(filtered_dataC['mainClass'] == option_selected) & (warrior["rarity"]=='common')]
        filtered_dataU = filtered_dataU[(filtered_dataU['mainClass'] == option_selected) & (warrior["rarity"]=='uncommon')] 
        filtered_dataR = filtered_dataR[(filtered_dataR['mainClass'] == option_selected) & (warrior["rarity"]=='rare')] 
        filtered_dataL = filtered_dataL[(filtered_dataL['mainClass'] == option_selected) & (warrior["rarity"]=='legendary')] 
        filtered_dataM = filtered_dataM[(filtered_dataM['mainClass'] == option_selected) & (warrior["rarity"]=='mythic')] 

    trace1 = go.Scatter(x=filtered_dataC.timeStamp, y=filtered_dataC.soldPrice, mode='markers', name='Common',
                            hovertemplate =
        '<b>ID</b>: %{text}<br>' +
        '<b>Price</b>: %{y} Jewels'+
        '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
         text = filtered_dataC['id'] + '<br>' + 
                '<b>Rarity</b>: ' + filtered_dataC['rarity'] + '<br>' + 
                '<b>Generation</b>: ' + filtered_dataC['generationStr'] + '<br>' + '<br>' + 
                '<b>Main Class</b>: ' + filtered_dataC['mainClass'] + '<br>' +
                '<b>Sub Class</b>: ' + filtered_dataC['subClass'] + '<br>' + 
                '<b>Primary Boost</b>: ' + filtered_dataC['statBoost1'] + '<br>' +
                '<b>Secondary Boost</b>: ' + filtered_dataC['statBoost2'] + '<br>' + 
                '<b>Profession</b>: ' + filtered_dataC['profession']+ '<br>',
         marker = dict(color='rgba(219, 217, 222, 1)', size=7)

                )

    trace2 = go.Scatter(x=filtered_dataU.timeStamp, y=filtered_dataU.soldPrice, mode='markers', name='Uncommon',
                            hovertemplate =
        '<b>ID</b>: %{text}<br>' +
        '<b>Price</b>: %{y} Jewels'+
        '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
         text = filtered_dataU['id'] + '<br>' + 
                '<b>Rarity</b>: ' + filtered_dataU['rarity'] + '<br>' +
                '<b>Generation</b>: ' + filtered_dataU['generationStr'] + '<br>' + '<br>' + 
                '<b>Main Class</b>: ' + filtered_dataU['mainClass'] + '<br>' +
                '<b>Sub Class</b>: ' + filtered_dataU['subClass'] + '<br>' + 
                '<b>Primary Boost</b>: ' + filtered_dataU['statBoost1'] + '<br>' +
                '<b>Secondary Boost</b>: ' + filtered_dataU['statBoost2'] + '<br>' + 
                '<b>Profession</b>: ' + filtered_dataU['profession']+ '<br>',
         marker = dict(color='rgba(115, 191, 131, 1)', size=7)
                )

    trace3 = go.Scatter(x=filtered_dataR.timeStamp, y=filtered_dataR.soldPrice, mode='markers', name='Rare',
                            hovertemplate =
        '<b>ID</b>: %{text}<br>' +
        '<b>Price</b>: %{y} Jewels'+
        '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
         text = filtered_dataR['id'] + '<br>' + 
                '<b>Rarity</b>: ' + filtered_dataR['rarity'] + '<br>' +
                '<b>Generation</b>: ' + filtered_dataR['generationStr'] + '<br>' + '<br>' + 
                '<b>Main Class</b>: ' + filtered_dataR['mainClass'] + '<br>' +
                '<b>Sub Class</b>: ' + filtered_dataR['subClass'] + '<br>' + 
                '<b>Primary Boost</b>: ' + filtered_dataR['statBoost1'] + '<br>' +
                '<b>Secondary Boost</b>: ' + filtered_dataR['statBoost2'] + '<br>' + 
                '<b>Profession</b>: ' + filtered_dataR['profession']+ '<br>',
        marker = dict(color='rgba(53, 147, 183, 1)', size=7)
                )

    trace4 = go.Scatter(x=filtered_dataL.timeStamp, y=filtered_dataL.soldPrice, mode='markers', name='Legendary',
                            hovertemplate =
        '<b>ID</b>: %{text}<br>' +
        '<b>Price</b>: %{y} Jewels'+
        '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
         text = filtered_dataL['id'] + '<br>' + 
                '<b>Rarity</b>: ' + filtered_dataL['rarity'] + '<br>' +
                '<b>Generation</b>: ' + filtered_dataL['generationStr'] + '<br>' + '<br>' + 
                '<b>Main Class</b>: ' + filtered_dataL['mainClass'] + '<br>' +
                '<b>Sub Class</b>: ' + filtered_dataL['subClass'] + '<br>' + 
                '<b>Primary Boost</b>: ' + filtered_dataL['statBoost1'] + '<br>' +
                '<b>Secondary Boost</b>: ' + filtered_dataL['statBoost2'] + '<br>' + 
                '<b>Profession</b>: ' + filtered_dataL['profession']+ '<br>',
        marker = dict(color='rgba(255, 164, 62, 1)', size=7)
                )

    trace5 = go.Scatter(x=filtered_dataM.timeStamp, y=warriorM.soldPrice, mode='markers', name='Mythic',
                            hovertemplate =
        '<b>ID</b>: %{text}<br>' +
        '<b>Price</b>: %{y} Jewels'+
        '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
         text = filtered_dataM['id'] + '<br>' + 
                '<b>Rarity</b>: ' + filtered_dataM['rarity'] + '<br>' +
                '<b>Generation</b>: ' + filtered_dataM['generationStr'] + '<br>' + '<br>' + 
                '<b>Main Class</b>: ' + filtered_dataM['mainClass'] + '<br>' +
                '<b>Sub Class</b>: ' + filtered_dataM['subClass'] + '<br>' + 
                '<b>Primary Boost</b>: ' + filtered_dataM['statBoost1'] + '<br>' +
                '<b>Secondary Boost</b>: ' + filtered_dataM['statBoost2'] + '<br>' + 
                '<b>Profession</b>: ' + filtered_dataM['profession']+ '<br>',
        marker = dict(color='rgba(178, 109, 216, 1)', size=7)          
                )

    data = [trace1, trace2, trace3, trace4, trace5]
    newfig = go.Figure(data=data)
    newfig.update_traces(marker=dict(line=dict(width=.5)))
    newfig.update_layout(title='Tavern Sales - Last 1000 Heroes Sold', 
                      titlefont=dict(family='Arial', size=24),
                      xaxis=dict(showgrid=True, ticks='outside'), 
                      xaxis_title='Date in UTC',
                      yaxis_title='Jewel',
                      plot_bgcolor='white'
                     )

    newfig.update_xaxes(showspikes=True)
    newfig.update_yaxes(showspikes=True)

    return newfig

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Nov/2021 21:56:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:15] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:15] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:16] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:16] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:16] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:16] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 21:56:16] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/